In [1]:
import cv2
print(cv2.cuda.getCudaEnabledDeviceCount())


0


In [3]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    print(f"CUDA is available. GPU: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available.")


CUDA is not available.


In [1]:
import socket

# Get the local IP address
local_ip = socket.gethostbyname(socket.gethostname())
print("Local IP address:", local_ip)


Local IP address: 192.168.10.13


In [2]:
def convert_to_cidr(ip):
    # Split the IP address by '.' to get the individual octets
    octets = ip.split('.')
    
    # Replace the last octet with 0 to get the network address
    network_address = '.'.join(octets[:3]) + '.0'
    
    # Return the network address with /24 CIDR
    return f"{network_address}/24"

# Example usage
local_ip = "192.168.10.13"
network_range = convert_to_cidr(local_ip)

print(f"Network in CIDR notation: {network_range}")


Network in CIDR notation: 192.168.10.0/24


In [3]:
import subprocess

# Get your local network range (for example, 192.168.10.0/24)
network_range = "192.168.10.0/24"  # Adjust this based on your network

# Run the Nmap scan using subprocess
nmap_command = f"nmap -sn {network_range}"
result = subprocess.run(nmap_command, shell=True, capture_output=True, text=True)

# Print the results
print("Scan results:")
print(result.stdout)

# Filter out IPs associated with "Hangzhou Hikvision Digital Technology"
filter_device = "80:BE:AF"
filtered_ips = []

# Process the nmap output to find the Hikvision IP
lines = result.stdout.splitlines()
for i in range(2, len(lines)):
    # Check if the current line contains the device manufacturer
    if filter_device in lines[i]:
        # The IP address is two lines before the manufacturer name
        ip_address = lines[i-2].split()[-1]  # IP is usually the last element in that line
        filtered_ips.append(ip_address)

# Display the filtered IP addresses
if filtered_ips:
    print("Hikvision IPs found:")
    for ip in filtered_ips:
        print(ip)
else:
    print("No Hikvision IP found.")


Scan results:
Starting Nmap 7.95 ( https://nmap.org ) at 2025-01-13 00:04 SE Asia Standard Time
Nmap scan report for wifi.cmcc (192.168.10.1)
Host is up (0.0040s latency).
MAC Address: D0:59:19:9C:3B:0D (zte)
Nmap scan report for 192.168.10.2 (192.168.10.2)
Host is up (0.053s latency).
MAC Address: CA:CE:D6:A5:6E:FF (Unknown)
Nmap scan report for 192.168.10.14 (192.168.10.14)
Host is up (0.0020s latency).
MAC Address: 80:BE:AF:AD:B3:0A (Hangzhou Hikvision Digital Technology)
Nmap scan report for 192.168.10.13 (192.168.10.13)
Host is up.
Nmap done: 256 IP addresses (4 hosts up) scanned in 2.22 seconds

Hikvision IPs found:
(192.168.10.14)


In [4]:

ip_address = str(filtered_ips[0])
ip_address = ip_address.strip("()")
print(ip_address)

192.168.10.14


In [5]:
import cv2
import socket
import time

# Target PC Details
TARGET_HOST = '160.22.122.122'
TARGET_PORT = 8000

# RTSP stream details
RTSP_ADDRESS = f"rtsp://admin:NohopeN3@{ip_address}:554/Streaming/Channels/102"

# Initialize video capture
cap = cv2.VideoCapture(RTSP_ADDRESS)

def reset_video_capture():
    """
    Reset the video capture stream if an error occurs.
    """
    global cap
    print("Resetting video capture...")
    cap.release()  # Release the old capture
    time.sleep(1)  # Allow the camera to reset
    cap = cv2.VideoCapture(RTSP_ADDRESS)  # Re-initialize
    if not cap.isOpened():
        print("Failed to re-open video capture.")
    return cap.isOpened()

def create_socket():
    """
    Creates and connects the socket. Returns the socket if successful.
    """
    try:
        client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        client_socket.connect((TARGET_HOST, TARGET_PORT))
        return client_socket
    except socket.error as e:
        print(f"Error creating socket: {e}")
        return None

def send_frame(client_socket, frame):
    """
    Sends a frame to the server via the socket.
    """
    try:
        # Encode the frame as JPEG
        _, jpeg = cv2.imencode('.jpg', frame)
        frame_bytes = jpeg.tobytes()

        # Send the frame length and data (only if socket is still valid)
        if client_socket and client_socket.fileno() != -1:  # Check if socket is open
            length = len(frame_bytes)
            client_socket.sendall(length.to_bytes(4, byteorder='big'))  # Length prefix
            client_socket.sendall(frame_bytes)
        else:
            print("Socket is not open. Reconnecting...")
            return False
    except Exception as e:
        print(f"Error while sending frame: {e}")
        return False
    return True

try:
    # Main loop for connection and video streaming
    while True:
        # 1. Create the socket
        client_socket = create_socket()
        if not client_socket:
            print("Connection failed, retrying in 5 seconds...")
            time.sleep(5)
            continue

        print("Connected to server, starting video stream.")
        
        try:
            while True:
                # 2. Read a frame from the video capture
                ret, frame = cap.read()

                if not ret:
                    print("Error reading frame, resetting video capture.")
                    if not reset_video_capture():
                        break  # Exit if unable to reinitialize the video stream
                    continue

                # 3. Send the frame
                if not send_frame(client_socket, frame):
                    print("Socket lost, attempting to reconnect...")
                    break  # Exit the inner loop to reconnect the socket

                # # Check for user input to exit
                # if cv2.waitKey(1) & 0xFF == ord('q'):
                #     print("Exiting on user request.")
                #     break

        except Exception as e:
            print(f"Error during video streaming: {e}")
        finally:
            # Clean up the socket connection
            if client_socket:
                client_socket.close()

        # # Outer loop exit condition
        # if cv2.waitKey(1) & 0xFF == ord('q'):
        #     break

finally:
    # Release resources
    if cap.isOpened():
        cap.release() 
    cv2.destroyAllWindows()
    print("Application ended.")

Connected to server, starting video stream.
Application ended.


KeyboardInterrupt: 